## 1: Importing the Dependencies

In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
!unzip /content/drive/MyDrive/DataScienceProjects/credit_card_fraud_detection/credit_card_fraud_detection.zip

Archive:  /content/drive/MyDrive/DataScienceProjects/credit_card_fraud_detection/credit_card_fraud_detection.zip
  inflating: creditcard.csv          


In [15]:
data = pd.read_csv("creditcard.csv")

In [16]:
# display the top 5 rows on our dataset
data.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [41]:
# dataset informations
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284807 entries, 0 to 284806
Data columns (total 31 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   Time    284807 non-null  float64
 1   V1      284807 non-null  float64
 2   V2      284807 non-null  float64
 3   V3      284807 non-null  float64
 4   V4      284807 non-null  float64
 5   V5      284807 non-null  float64
 6   V6      284807 non-null  float64
 7   V7      284807 non-null  float64
 8   V8      284807 non-null  float64
 9   V9      284807 non-null  float64
 10  V10     284807 non-null  float64
 11  V11     284807 non-null  float64
 12  V12     284807 non-null  float64
 13  V13     284807 non-null  float64
 14  V14     284807 non-null  float64
 15  V15     284807 non-null  float64
 16  V16     284807 non-null  float64
 17  V17     284807 non-null  float64
 18  V18     284807 non-null  float64
 19  V19     284807 non-null  float64
 20  V20     284807 non-null  float64
 21  V21     28

## 2: Checking the missing values

In [18]:
# checking the number of missing values in each column
data.isnull().sum()

Time      0
V1        0
V2        0
V3        0
V4        0
V5        0
V6        0
V7        0
V8        0
V9        0
V10       0
V11       0
V12       0
V13       0
V14       0
V15       0
V16       0
V17       0
V18       0
V19       0
V20       0
V21       0
V22       0
V23       0
V24       0
V25       0
V26       0
V27       0
V28       0
Amount    0
Class     0
dtype: int64

## 3: checking if target variable is balanced or not

In [19]:
# distribution of legit transactions & fraudulent transactions
data['Class'].value_counts()

Class
0    284315
1       492
Name: count, dtype: int64

as we can see our target variable is impalanced.

0 : Normal Transaction

1 : fraudulent transaction

In [20]:
# seperating the two categories that are in our target variable.
legit = data[data.Class == 0]
fraud = data[data.Class == 1]

In [21]:
# how many rows and columns on legit.
print(legit.shape)
# how many rows and colmns on fraud.
print(fraud.shape)

(284315, 31)
(492, 31)


In [22]:
# statistical measures of the data
legit.Amount.describe()

count    284315.000000
mean         88.291022
std         250.105092
min           0.000000
25%           5.650000
50%          22.000000
75%          77.050000
max       25691.160000
Name: Amount, dtype: float64

In [23]:
fraud.Amount.describe()

count     492.000000
mean      122.211321
std       256.683288
min         0.000000
25%         1.000000
50%         9.250000
75%       105.890000
max      2125.870000
Name: Amount, dtype: float64

In [24]:
# compare the values for both transactions
data.groupby('Class').mean()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
Class,,,,,,,,,,,,,,,,,,,,,
0,94838.202258,0.008258,-0.006271,0.012171,-0.007860,0.005453,0.002419,0.009637,-0.000987,0.004467,...,-0.000644,-0.001235,-0.000024,0.000070,0.000182,-0.000072,-0.000089,-0.000295,-0.000131,88.291022
1,80746.806911,-4.771948,3.623778,-7.033281,4.542029,-3.151225,-1.397737,-5.568731,0.570636,-2.581123,...,0.372319,0.713588,0.014049,-0.040308,-0.105130,0.041449,0.051648,0.170575,0.075667,122.211321


## 4: Under-Sampling

Build a sample dataset containing similar distribution of normal transactions and Fraudulent Transactions

Number of Fraudulent Transactions --> 492

In [25]:
# legit contains a 284315, so we need to make it 492 like the fraud category
legit_sample = legit.sample(n=492)

Concatenating two DataFrames

In [26]:
# now we have balanced dataset, for legit_sample 492 and fraud for 492
new_dataset = pd.concat([legit_sample, fraud], axis=0)

In [27]:
new_dataset.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
245826,152926.0,-1.082285,0.266806,-0.850345,-1.592843,0.203306,-0.076924,1.511204,-0.044068,-1.813778,...,-0.309350,-0.495283,-0.232440,0.127221,0.315434,0.264848,-0.072347,-0.061572,212.00,0
100901,67661.0,1.310183,-1.179386,1.094067,-0.642661,-1.095036,1.662197,-1.758332,0.588463,0.533829,...,0.135272,0.751543,-0.237144,-1.196195,0.549863,0.046418,0.096660,0.001138,7.20,0
167941,118975.0,-0.540097,-0.794502,-2.310880,-2.189124,0.097549,-2.039749,0.620583,-0.465602,-2.592446,...,0.363603,1.578547,-0.385782,0.106929,-1.333706,-0.085334,0.189632,-0.017113,112.51,0
115259,73808.0,0.617897,-1.195194,1.904863,1.927988,-1.865795,0.909455,-1.114847,0.503209,1.679076,...,0.266276,0.766720,-0.319932,0.590897,0.427207,-0.148653,0.076132,0.068176,216.00,0
23645,32851.0,1.231885,0.174864,0.109255,0.870344,0.353796,0.555864,-0.131860,0.091758,0.168240,...,-0.127402,-0.333191,-0.130451,-1.334582,0.532116,-0.365689,0.047509,0.018100,23.00,0


In [28]:
# our target variable contains a balanced 2 categories which are 0 and 1
new_dataset['Class'].value_counts()

Class
0    492
1    492
Name: count, dtype: int64

now we have balanced target variable that contins 492 for each.

In [29]:
# group the data by using the target variable(Class)
new_dataset.groupby('Class').mean()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
Class,,,,,,,,,,,,,,,,,,,,,
0,96622.158537,-0.028676,-0.150287,-0.050480,-0.001293,0.069732,0.011570,0.039684,0.035571,-0.016334,...,0.040170,0.009311,-0.062800,-0.029199,-0.010949,-0.024956,-0.007918,0.002291,0.004079,111.698598
1,80746.806911,-4.771948,3.623778,-7.033281,4.542029,-3.151225,-1.397737,-5.568731,0.570636,-2.581123,...,0.372319,0.713588,0.014049,-0.040308,-0.105130,0.041449,0.051648,0.170575,0.075667,122.211321


## 5: Splitting the data into Features & Targets

In [30]:
# X contains all the variables except the target variable.
X = new_dataset.drop(columns='Class', axis=1)
# Y contains only the target variable.
Y = new_dataset['Class']

In [31]:
# displaying the values of X
print(X)

            Time        V1        V2        V3        V4        V5        V6  \
245826  152926.0 -1.082285  0.266806 -0.850345 -1.592843  0.203306 -0.076924   
100901   67661.0  1.310183 -1.179386  1.094067 -0.642661 -1.095036  1.662197   
167941  118975.0 -0.540097 -0.794502 -2.310880 -2.189124  0.097549 -2.039749   
115259   73808.0  0.617897 -1.195194  1.904863  1.927988 -1.865795  0.909455   
23645    32851.0  1.231885  0.174864  0.109255  0.870344  0.353796  0.555864   
...          ...       ...       ...       ...       ...       ...       ...   
279863  169142.0 -1.927883  1.125653 -4.518331  1.749293 -1.566487 -2.010494   
280143  169347.0  1.378559  1.289381 -5.004247  1.411850  0.442581 -1.326536   
280149  169351.0 -0.676143  1.126366 -2.213700  0.468308 -1.120541 -0.003346   
281144  169966.0 -3.113832  0.585864 -5.399730  1.817092 -0.840618 -2.943548   
281674  170348.0  1.991976  0.158476 -2.583441  0.408670  1.151147 -0.096695   

              V7        V8        V9  .

In [32]:
# displaying the values of Y
print(Y)

245826    0
100901    0
167941    0
115259    0
23645     0
         ..
279863    1
280143    1
280149    1
281144    1
281674    1
Name: Class, Length: 984, dtype: int64


Split the data into Training data & Testing Data

In [33]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, stratify=Y, random_state=2)

In [34]:
print(X.shape, X_train.shape, X_test.shape)

(984, 30) (787, 30) (197, 30)


## 6: Model Training

Logistic Regression

In [35]:
model = LogisticRegression()

In [36]:
# training the Logistic Regression Model with Training Data
model.fit(X_train, Y_train)

LogisticRegression()

## 7: Model Evaluation

Accuracy Score

In [37]:
# accuracy on training data
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction, Y_train)

In [42]:
print('Accuracy on Training data : ', training_data_accuracy)

Accuracy on Training data :  0.9453621346886912


In [39]:
# accuracy on test data
X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(X_test_prediction, Y_test)

In [40]:
print('Accuracy score on Test Data : ', test_data_accuracy)

Accuracy score on Test Data :  0.934010152284264


good, we trained a model using logistic regression for 94%  
also the test dataset for 93%, which is good percentage.